## BRB Practical 1 Final

### Organizing the Data
Import the necessary libraries, read in the csvs, create the Xtraining, Ytraining, and Xtest sets

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import warnings
warnings.filterwarnings("ignore")
import math

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train_full = pd.read_csv("./train.csv")
df_test_full = pd.read_csv("./test.csv")

In [14]:
df_test = df_test_full
df_train = df_train_full

In [15]:
### Swap this out in the real thing!!! 
df_train = df_train_full.sample(n=100000)

In [16]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
442682,[nH]1cccc1-c1cc2[nH]c3c4[SiH2]C=Cc4c4cocc4c3c2cn1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.87
605414,C1=Cc2c([SiH2]1)c1[se]c3cc(-c4nccc5nsnc45)c4ns...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.33
71410,c1ccc(cn1)-c1ncc(-c2ccc(-c3cncc4nsnc34)c3nsnc2...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.90
588589,C1=Cc2cc3c(cc2[SiH2]1)c1sc(cc1c1=C[SiH2]C=c31)...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.78
753820,c1[SiH2]c2c3-[s]c4ccc5cocc5c4-c3c3cCcc3c2-c1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68


In [17]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [19]:
test_ids = df_test.index
df_test = df_test.drop(['Id'], axis=1)

### Feature engineering

In [20]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
442682,[nH]1cccc1-c1cc2[nH]c3c4[SiH2]C=Cc4c4cocc4c3c2cn1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
605414,C1=Cc2c([SiH2]1)c1[se]c3cc(-c4nccc5nsnc45)c4ns...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71410,c1ccc(cn1)-c1ncc(-c2ccc(-c3cncc4nsnc34)c3nsnc2...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
588589,C1=Cc2cc3c(cc2[SiH2]1)c1sc(cc1c1=C[SiH2]C=c31)...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
753820,c1[SiH2]c2c3-[s]c4ccc5cocc5c4-c3c3cCcc3c2-c1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)
#Drop the 'smiles' column
# df_all = df_all.drop(['smiles'], axis=1)

In [21]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (100000, 256)
Train gap: (100000,)
Test features: (824230, 256)


### Creating simple predictors, evaluate

Here we use our training set to train Linear Regression Models, Random Forest, Random Forest with 100 trees, Bayesian Ridge Regression, and Lasso models. 

Once we select the best model, we'll train on the entire data set and predict for the test values.
Below, we evaluate our simple predictors with root mean squared error and 5-fold cross validation (printed under each model). At a high level, we can see that random forest with 100 trees does the best.

In [22]:
LR = LinearRegression()
print "Simple Linear Regression Model"
print cross_val_score(LR, X_train, Y_train, cv=5, scoring="mean_squared_error")

Simple Linear Regression Model
[ -8.93540075e-02  -8.85427904e-02  -4.61132662e+20  -8.91306503e-02
  -8.94801136e-02]


In [23]:
RF = RandomForestRegressor()
print "Random Forest Regressor"
print cross_val_score(RF, X_train, Y_train, cv=5, scoring="mean_squared_error")

Random Forest Regressor
[-0.0757211  -0.07507246 -0.07644199 -0.07635965 -0.0761524 ]


Eventually, we will optimize the hyper parameters we pass into these functions

In [24]:
RF_100 = RandomForestRegressor(n_estimators=100)
print "Random Forest Regressor with 100 Trees"
print cross_val_score(RF_100, X_train, Y_train, cv=5, scoring="mean_squared_error")

Random Forest Regressor with 100 Trees
[-0.0756296  -0.074949   -0.07638335 -0.07623079 -0.07604093]


In [25]:
# Bayesian Ridge Regression
BRR = BayesianRidge(compute_score=True)
print cross_val_score(BRR, X_train, Y_train, cv=5, scoring="mean_squared_error")

[-0.08934134 -0.08855736 -0.09030493 -0.08911527 -0.08947525]


In [26]:
# Lasso Regression
L = Lasso(alpha=0.1)
print cross_val_score(L, X_train, Y_train, cv=5, scoring="mean_squared_error")

[-0.16538462 -0.16626154 -0.16476814 -0.16641375 -0.16572559]


### Optimizing the Hyper parameters
#### Grid Search

### Ensemble Methods

Inspired by the ensemble methods like random forest which take an average of the predictions by each tree, we wanted to combine the predictions of a divers model. 

Here we add a new feature for our linear model predictions, random forest model predictions, lasso model predictions, etc. This becomes our new training set.

#### Creating our training set

In [28]:
# fit all the models on the training set so we can start to make predictions
LR.fit(X_train, Y_train)
RF.fit(X_train, Y_train)
RF_100.fit(X_train, Y_train)
BRR.fit(X_train, Y_train)
L.fit(X_train, Y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [36]:
# make the predictions on our training set so we can add them as features
LR_pred_train = LR.predict(X_train)
RF_pred_train = RF.predict(X_train)
RF_pred_100_train = RF_100.predict(X_train)
BRR_pred_train = BRR.predict(X_train)
L_pred_train = L.predict(X_train)

# create a new data frame for what we call the "ensemble" method
df_train_ensemble = df_train.drop(['smiles'], axis=1)

In [40]:
# add the feature vectors for the predictions
df_train_ensemble['LR_pred'] = LR_pred_train
df_train_ensemble['RF_pred'] = RF_pred_train
df_train_ensemble['RF_pred_100'] = RF_pred_100_train
df_train_ensemble['BRR_pred'] = BRR_pred_train
df_train_ensemble['L_pred'] = L_pred_train

In [41]:
df_train_ensemble.head(5)

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,feat_252,feat_253,feat_254,feat_255,feat_256,LR_pred,RF_pred,RF_pred_100,BRR_pred,L_pred
442682,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.997156,2.064967,2.062232,1.997794,1.913956
605414,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.641202,1.621703,1.627049,1.641357,1.913956
71410,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.740283,1.706781,1.708116,1.740084,1.913956
588589,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.744102,1.786424,1.788058,1.743993,1.913956
753820,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.966072,1.895954,1.890638,1.965579,1.913956


In [42]:
X_train_ensemble = df_train_ensemble.values
print "Train features:", X_train_ensemble.shape
print "Train gap:", Y_train.shape
print "Y train is the same as before since the actual values never changed"

Train features: (100000, 261)
Train gap: (100000,)
Y train is the same as before since the actual values never changed


### Evaluating our new model
We can now perform linear regression (and other models) on our new training set that includes the predictions and see how it performs

In [44]:
LR_ensemble = LinearRegression()
print "Linear Regression Model including Predictions as Features"
print cross_val_score(LR_ensemble, X_train_ensemble, Y_train, cv=5, scoring="mean_squared_error")

Linear Regression Model including Predictions as Features
[ -7.26958606e-02  -7.21181915e-02  -1.84624015e+20  -7.30851521e-02
  -7.31680295e-02]


In [45]:
RF_ensemble = RandomForestRegressor()
print "Random Forest Model including Predictions as Features"
print cross_val_score(RF_ensemble, X_train_ensemble, Y_train, cv=5, scoring="mean_squared_error")

Random Forest Model including Predictions as Features
[-0.07510553 -0.07423559 -0.07595545 -0.0756203  -0.07552642]


In [88]:
RF_100_ensemble = RandomForestRegressor(n_estimators=100)
print "Random Forest Model including Predictions as Features"
print cross_val_score(RF_100_ensemble, X_train_ensemble, Y_train, cv=5, scoring="mean_squared_error")

Random Forest Model including Predictions as Features
[-0.07495048 -0.07423109 -0.07587937 -0.07551595 -0.07536452]


In [46]:
BRR_ensemble = BayesianRidge(compute_score=True)
print "Bayesian Ridge Regression including Predictions as Features"
print cross_val_score(BRR_ensemble, X_train_ensemble, Y_train, cv=5, scoring="mean_squared_error")

Bayesian Ridge Regression including Predictions as Features
[-0.07268796 -0.07211191 -0.07364589 -0.07308256 -0.07315892]


In [47]:
L_ensemble = Lasso(alpha=0.1)
print "Lasso Regression including Predictions as Features"
print cross_val_score(L_ensemble, X_train_ensemble, Y_train, cv=5, scoring="mean_squared_error")

Lasso Regression including Predictions as Features
[-0.16538462 -0.16626154 -0.16476814 -0.16641375 -0.16572559]


It seems doing linear regression on the new training set does perform better than our baselines. However--this is not an altogether fair test since our original simple prediction models were fitted on the training set. Then predicting on the training set therefore, we expect to be much more accurate and relevant.

In [89]:
# Fit the models for future use
LR_ensemble.fit(X_train_ensemble, Y_train)
RF_ensemble.fit(X_train_ensemble, Y_train)
RF_100_ensemble.fit(X_train_ensemble, Y_train)
BRR_ensemble.fit(X_train_ensemble, Y_train)
L_ensemble.fit(X_train_ensemble, Y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

### Further Comparison of methods

We construct a test set 100,000 data points that did not overlap with our original 100,000 data points in the training set. 

In [65]:
df_test = df_train_full.sample(n=100000)
# remove test values that were in the training set
df_test['overlap'] = df_test.index.isin(df_train.index)
df_test = df_test[df_test['overlap'] == 0]
df_test = df_test.drop(['smiles'], axis=1)
df_test = df_test.drop(['overlap'], axis=1)
df_test.head(5)

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
490311,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.33
664466,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.98
482879,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.29
927394,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.30
170643,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.69


In [66]:
Y_test = df_test.gap.values

In [78]:
# we call this X_test_ensemble because we created this new test to evaluate the algorithms including ensemble model
X_test_new = df_test.drop(['gap'], axis=1)
X_test_new = X_test_new.values

#### Predictions of the simple models on the new set

In [79]:
LR_pred_test = LR.predict(X_test_new)
BRR_pred_test = BRR.predict(X_test_new)
RF_pred_test = RF.predict(X_test_new)
RF_pred_100_test = RF_100.predict(X_test_new)
L_pred_test = L.predict(X_test_new)

#### Create new test set for ensemble methods and predictions of ensemble models on the new set

In [82]:
### Create a test set for the ensemble method
X_test_ensemble = df_test.drop(['gap'], axis=1)
X_test_ensemble['LR_pred'] = LR_pred_test
X_test_ensemble['RF_pred'] = RF_pred_test
X_test_ensemble['RF_pred_100'] = RF_pred_100_test
X_test_ensemble['BRR_pred'] = BRR_pred_test
X_test_ensemble['L_pred'] = L_pred_test

In [86]:
X_test_ensemble = X_test_ensemble.values

In [90]:
LR_pred_test_ensemble = LR_ensemble.predict(X_test_ensemble)
BRR_pred_test_ensemble = BRR_ensemble.predict(X_test_ensemble)
RF_pred_test_ensemble = RF_ensemble.predict(X_test_ensemble)
RF_pred_100_test_ensemble = RF_100_ensemble.predict(X_test_ensemble)
L_pred_test_ensemble = L_ensemble.predict(X_test_ensemble)

#### Evaluate all the methods through root mean squared error

In [93]:
# baselines Linear
LR_rms = math.sqrt(mean_squared_error(LR_pred_test, Y_test))
print "LR_rms"
print LR_rms
#baselines Random Forest
RF_rms = math.sqrt(mean_squared_error(RF_pred_test, Y_test))
print "RF_rms"
print RF_rms
BRR_rms = math.sqrt(mean_squared_error(BRR_pred_test, Y_test))
print "BRR_rms"
print BRR_rms
# lasso regression
L_rms = math.sqrt(mean_squared_error(L_pred_test, Y_test))
print "L_rms"
print L_rms
# random forest with 100
RF_100_rms = math.sqrt(mean_squared_error(RF_pred_100_test, Y_test))
print "RF_100_rms"
print RF_100_rms

LR_rms
0.298618539898
RF_rms
0.275096230434
BRR_rms
0.298621208601
L_rms
0.405979583506
RF_100_rms
0.274812554882


In [95]:
# ensemble methodx
# linear ensemble
LR_ensemble_rms = math.sqrt(mean_squared_error(LR_pred_test_ensemble, Y_test))
print "LR_ensemble_rms"
print LR_ensemble_rms
# random forest with 
RF_ensemble_rms = math.sqrt(mean_squared_error(RF_pred_test_ensemble, Y_test))
print "RF_ensemble_rms"
print RF_ensemble_rms
# random forest with 
RF_ensemble_rms_100 = math.sqrt(mean_squared_error(RF_pred_test_ensemble, Y_test))
print "RF_ensemble_rms_100"
print RF_ensemble_rms_100
# bayesian ensemble
BRR_ensemble_rms = math.sqrt(mean_squared_error(BRR_pred_test_ensemble, Y_test))
print "BRR_ensemble_rms"
print BRR_ensemble_rms
# lasso ensemble
L_ensemble_rms = math.sqrt(mean_squared_error(RF_pred_100_test_ensemble, Y_test))
print "L_ensemble_rms"
print L_ensemble_rms

LR_ensemble_rms
0.27484839019
RF_ensemble_rms
0.275236910275
RF_ensemble_rms_100
0.275236910275
BRR_ensemble_rms
0.274842213239
L_ensemble_rms
0.275135594831


### Conclusion
In conclusion, it seems like the ensemble method works, but we might have better success after more optimizing of the hyper parameters in each model.

#### Evaluate

In [ ]:
# baselines Linear
LR_rms = math.sqrt(mean_squared_error(LR_pred, ensemble_test_gap))
print "LR_rms"
print LR_rms

#baselines Random Forest
RF_rms = math.sqrt(mean_squared_error(RF_pred, ensemble_test_gap))
print "RF_rms"
print RF_rms

So for some reason this is even worse :( Maybe should keep all the old features as well...

### Save results to CSV

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)
write_to_file("bayesianridge.csv", BRR_pred)
write_to_file("randomforest100trees.csv", RF_pred_100)